<div style="text-align:center;font-size:22pt; font-weight:bold;color:white;border:solid black 1.5pt;background-color:#1e7263;">
    Understanding Model History Object: Classification Task
</div>

In [1]:
# ======================================================================= #
# Course: Deep Learning Complete Course (CS-501)
# Author: Dr. Saad Laouadi
# Institution: Quant Coding Versity Academy
# Date: December 25, 2024
#
# ==========================================================
# Lesson: Understanding Model History Object in Keras
#         Analyzing and Visualizing Training Progress
# ==========================================================
# ## Learning Objectives
# This guide will enable you to:
# 1. Access and interpret the model.fit() history object
# 2. Extract and analyze training metrics over epochs
# 3. Visualize training and validation metrics
# 4. Identify optimal training epochs and model performance
# 5. Detect overfitting through history analysis
# =======================================================================
#          Copyright © Dr. Saad Laouadi 2024
# =======================================================================

In [21]:
# ==================================================== #
#        Load Required Libraries
# ==================================================== #

import os  
import shutil
import json

# Disable Metal API Validation
os.environ["METAL_DEVICE_WRAPPER_TYPE"] = "0"  


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
 
                          

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn import metrics, preprocessing

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    precision_recall_curve
)


# import tensorflow
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# Set styling for better visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("="*72)

%reload_ext watermark
%watermark -a "Dr. Saad Laouadi" -u -d -m

print("="*72)
print("Imported Packages and Their Versions:")
print("="*72)

%watermark -iv
print("="*72)

# Global Config
RANDOM_STATE = 101

Author: Dr. Saad Laouadi

Last updated: 2024-12-30

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Imported Packages and Their Versions:
json      : 2.0.9
pandas    : 2.2.2
tensorflow: 2.16.2
sklearn   : 1.5.1
matplotlib: 3.9.2
numpy     : 1.26.4
seaborn   : 0.13.2
keras     : 3.6.0



In [23]:
def cleanup_directory(directory_path):
    """
    Deletes the specified directory and all its contents.

    Args:
        directory_path (str): Path to the directory to delete.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        shutil.rmtree(directory_path)
        print(f"Directory '{os.path.basename(directory_path)}' deleted successfully.")
    else:
        print(f"Directory '{os.path.basename(directory_path)}' does not exist or is not a directory.")

### ML Model Raodmap
1. Read the data
2. Explore the data

3. Processing
    - Numerical features (scaling)
    
4. Splitting the data

In [6]:
# ==================================================== #
#        Implementing ModelCheckpoint 
#        Callback with Synthetic data
# ==================================================== #

In [30]:
# Set random seeds for reproducibility
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

# Create synthetic dataset
def create_synthetic_data(n_samples=1000):
    X, y = make_classification(
        n_samples=n_samples,
        n_features=20,
        n_informative=15,
        n_redundant=5,
        random_state=RANDOM_STATE
    )
    
    # One-hot encode the target
    y = tf.keras.utils.to_categorical(y)
    
    return train_test_split(X, y, test_size=0.2, random_state=42)


# Create a simple neural network model
def create_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape = input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [31]:
# Generate synthetic data
X_train, X_test, y_train, y_test = create_synthetic_data()
print(f"The X_train shape: {X_train.shape}")
print(f"The X_test shape: {X_test.shape}")
print(f"The y_train shape: {y_train.shape}")
print(f"The y_test shape:  {y_test.shape}")

The X_train shape: (800, 20)
The X_test shape: (200, 20)
The y_train shape: (800, 2)
The y_test shape:  (200, 2)


In [32]:
# Create model
model = create_model((X_train.shape[1],))

In [33]:
# Create checkpoint directory if it doesn't exist
checkpoint_dir = 'model_checkpoints'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
    print(f"Directory {os.path.basename(checkpoint_dir)} created successfully")
else:
    print("Directory already exists")

Directory model_checkpoints created successfully


In [34]:
# Define different types of ModelCheckpoint callbacks

# 1. Save the best model based on validation accuracy
checkpoint_best = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'best_model.keras'),
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)
# Train the model with callbacks
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint_best],
    verbose=1
)

Epoch 1/20
14/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5567 - loss: 0.8394
Epoch 1: val_accuracy improved from -inf to 0.76875, saving model to model_checkpoints/best_model.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5836 - loss: 0.7968 - val_accuracy: 0.7688 - val_loss: 0.4913
Epoch 2/20
15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7785 - loss: 0.4982
Epoch 2: val_accuracy improved from 0.76875 to 0.83125, saving model to model_checkpoints/best_model.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7876 - loss: 0.4830 - val_accuracy: 0.8313 - val_loss: 0.3913
Epoch 3/20
15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8476 - loss: 0.3826
Epoch 3: val_accuracy improved from 0.83125 to 0.85000, saving model to model_checkpoints/best_model.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8524 - loss: 0.3719 - val_accuracy: 0.8500 - val_loss: 0.3394
Epoch 4/20
15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8859 - loss: 0.311

In [35]:
# 2. Save the best model based on validation loss
checkpoint_min_loss = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'min_loss_model.keras'),
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

# Train the model with callbacks
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint_min_loss],
    verbose=1
)

Epoch 1/20
15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9974 - loss: 0.0464
Epoch 1: val_loss improved from inf to 0.11938, saving model to model_checkpoints/min_loss_model.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9971 - loss: 0.0444 - val_accuracy: 0.9438 - val_loss: 0.1194
Epoch 2/20
15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0419
Epoch 2: val_loss improved from 0.11938 to 0.11522, saving model to model_checkpoints/min_loss_model.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0400 - val_accuracy: 0.9438 - val_loss: 0.1152
Epoch 3/20
15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0374
Epoch 3: val_loss improved from 0.11522 to 0.11098, saving model to model_checkpoints/min_loss_model.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0358 - val_accuracy: 0.9438 - val_loss: 0.1110
Epoch 4/20
15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0336

In [26]:
# 3. Save models periodically
checkpoint_periodic = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}.keras'),
    save_freq=5,  # Save every 5 epochs
    verbose=1
)
# Train the model with callbacks
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint_periodic],
    verbose=1
)

In [36]:
# Train the model with callbacks
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint_best, checkpoint_min_loss, checkpoint_periodic],
    verbose=1
)

Epoch 1/20
 1/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.0123
Epoch 1: saving model to model_checkpoints/model_epoch_01.keras
 6/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.0089
Epoch 1: saving model to model_checkpoints/model_epoch_01.keras
11/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.0085
Epoch 1: saving model to model_checkpoints/model_epoch_01.keras
16/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.0080
Epoch 1: saving model to model_checkpoints/model_epoch_01.keras

Epoch 1: val_accuracy improved from 0.94375 to 0.96875, saving model to model_checkpoints/best_model.keras

Epoch 1: val_loss improved from 0.07521 to 0.07435, saving model to model_checkpoints/min_loss_model.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0077 - val_accuracy: 0.9688 - val_loss: 0.0743
Epoch 2/20
 1/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0116
Epoch 2: saving mode

In [37]:
# Call the cleanup function at the end of your notebook
cleanup_directory(checkpoint_dir)

Directory 'model_checkpoints' deleted successfully.
